In [1]:
###################################LoadAllTheSHit

import torch

import numpy as np
import torchvision
from torchvision import datasets, models, transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time

from PIL import Image



image_path = 'C:/PyTorch/flower_data/test/78/image_01874.jpg'

# check if CUDA is available
train_on_gpu = False

#########################################

model = models.vgg16(pretrained=False)

for param in model.features.parameters():
    param.requires_grad = False
n_inputs = model.classifier[6].in_features

last_layer = nn.Linear(n_inputs, 102)
model.classifier[6] = last_layer

criterion = nn.CrossEntropyLoss()

def load_checkpoint(filepath):
    checkpoint = torch.load(filepath, map_location='cpu')

    model.load_state_dict(checkpoint['state_dict'], strict=False)
    #model.load_state_dict(torch.load(filepath, map_location='cpu'), strict=False)
    
    model.class_to_idx = checkpoint['class_to_idx']

    return model

model = load_checkpoint('BAMFFlowerPowerRevDv2True.pth')


#########################

def process_image(image_path):
    ''' 
    Scales, crops, and normalizes a PIL image for a PyTorch       
    model, returns an Numpy array
    '''
    # If you used something other than 224x224 cropped images, set the correct size here
    #image_size = 224
    # Values you used for normalizing the images. Default here are for 
    # pretrained models from torchvision.
    #norm_mean = [0.485, 0.456, 0.406]
    #norm_std = [0.229, 0.224, 0.225]
    # Open the image
    img = Image.open(image_path)

    # Resize
    if img.size[0] > img.size[1]:
        img.thumbnail((50000, 256))
    else:
        img.thumbnail((256, 50000))

    # Crop 
    left_margin = (img.width-224)/2
    bottom_margin = (img.height-224)/2
    right_margin = left_margin + 224
    top_margin = bottom_margin + 224
    img = img.crop((left_margin, bottom_margin, right_margin, top_margin))
    
    # Normalize
    img = np.array(img)/255
    mean = np.array([0.485, 0.456, 0.406]) #Required mean
    std = np.array([0.229, 0.224, 0.225]) #Required std
    img = (img - mean)/std
    
    # Move color channels PyTorch expects the color channel to be the first dimension
    #img = img.numpy().transpose((2, 0, 1))
    img = img.transpose((2, 0, 1))
    
    return img
########################################################
   
## doneabove

def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

##################################################

def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file
    # Process image
    img = process_image(image_path)    
    # Numpy -> Tensor
    image_tensor = torch.from_numpy(img).type(torch.FloatTensor)
    # Add batch of size 1 to image
    model_input = image_tensor.unsqueeze(0)    
    # Probs
    probs = torch.exp(model.forward(model_input))    
    # Top probs
    top_probs, top_labs = probs.topk(topk)
    top_probs_sum = np.sum(top_probs.detach().numpy())
    top_probs = top_probs.detach().numpy().tolist()[0] 
    top_labs = top_labs.detach().numpy().tolist()[0]    
    # Convert indices to classes
    idx_to_class = {val: key for key, val in model.class_to_idx.items()}
    top_labels = [idx_to_class[lab] for lab in top_labs]

    
    return top_probs/top_probs_sum, top_labels

#######################################

probs, classes = predict(image_path, model.cpu())

print(probs)
print(classes)

[1.00000000e+00 2.19701077e-08 9.45304756e-09 8.51529577e-09
 2.69740258e-10]
['78', '91', '88', '39', '73']
